# 說明

原本想說做成搶課程式，不過我不是大學生，這個東西就隨意ㄅ

In [7]:
import requests
import bs4

from IPython.display import HTML
from PIL import Image
import io

def renderAsHtml(content:bytes):
    display(HTML(str(content, "utf8")))
    ()

userid = "" # NCKU 帳號
passwd = "" # Passwd 密碼

In [8]:
ncku_forcelogin = "https://course.ncku.edu.tw/index.php?c=auth&m=force_login"
ncku_logout = "https://course.ncku.edu.tw/index.php?c=auth&m=logout"
ncku_student_info = "https://course.ncku.edu.tw/index.php?c=profile&time=1659426250" # time is current time.
ncku_get = "https://course.ncku.edu.tw/index.php?c=auth"
ncku_save_session = "https://course.ncku.edu.tw/index.php?c=qry11215&m=save_sess"
ncku_course_schedule = "https://course.ncku.edu.tw/index.php?c=cos32315" # 我的課表

# 登入

In [9]:
r = requests.get(ncku_get)

# 登入用的資料是寫在網頁上，透過 submit 提交 Http POST
soap = bs4.BeautifulSoup(r.content)
# NCKU 把 csfrtoken 放在網頁上
csfrtoken = soap.find("form", {"id" : "LoginForm"}).find("input", {"name": "csrftoken"})["value"]

# COURSE_WEB 跟 PHPSESSID 是必要欄位
# 根據 Google, PHPSESSID 是 PHP 伺服器給的 session 是必要欄位
cookies = r.cookies.get_dict()

print(cookies)
print(csfrtoken)

{'COURSE_WEB': 'ffffffff8f7cbb0045525d5f4f58455e445a4a423660', 'PHPSESSID': '7999f8ceb173240c26e88ca65840add7'}
U2hWKAJ3WjBXfwU9BWxQYVdiAyYFLFd3CDJRa1RpXHQBIlVmVCQBcFVmBHYEcVt0AyUBbl8mUTdQYVAuBTs=


In [10]:
# 驗證碼，看起來是用 imagstick 產生的。
ncku_verifycode = "https://course.ncku.edu.tw/index.php?c=verifycode"
r = requests.get(ncku_verifycode, cookies=cookies)

image = Image.open(io.BytesIO(r.content))
display(image)
print(r.cookies.get_dict())
print(cookies)

{'COURSE_WEB': 'ffffffff8f7cbb0045525d5f4f58455e445a4a423660'}
{'COURSE_WEB': 'ffffffff8f7cbb0045525d5f4f58455e445a4a423660', 'PHPSESSID': '7999f8ceb173240c26e88ca65840add7'}


In [ ]:
ncku_url = "https://course.ncku.edu.tw/index.php?c=auth&m=login"
payload = {
    "user_id" : userid, # 帳號
    "passwd" : passwd,  # 密碼
    "code" : "0811",
    "csrftoken" : csfrtoken
}
r = requests.post(ncku_url, data=payload, cookies=cookies, allow_redirects=True)

cookies = r.history[0].cookies.get_dict()

# PHPSESSID 會被設定成與學號相關的字串，這樣就是完成登入了。
print(cookies)

# 我的課表

In [14]:
r = requests.get(ncku_course_schedule, cookies=cookies)
soap = bs4.BeautifulSoup(r.content)
table = soap.find("table", {"id": "cl-table"})

In [21]:
tt = table.__str__()

display(HTML(tt))